### This genetic algorithm is an adaptation of [MourvanZhou's evolutionary algorithm code](https://github.com/MorvanZhou/Evolutionary-Algorithm/blob/master/tutorial-contents/Genetic%20Algorithm/Genetic%20Algorithm%20Basic.py) ###

In [20]:
from qutip import *
from scipy import arcsin, sqrt, pi, sin, cos
import numpy as np
import scipy.sparse as sp
from qutip.qobj import Qobj
%matplotlib
import matplotlib.pyplot as plt
import itertools
import copy
from math import log2
from scipy.optimize import minimize
from IPython.core.display import clear_output

Using matplotlib backend: Qt5Agg


In [21]:
INPUT_STATES = 8 # at most 16

In [22]:
def h(x):  #binary entropy
    return -x*log2(x) - (1-x)*log2(1-x)

def calc_XN(p):
    def calc_XN_tmp(q):
        tmp = h(q * (1-p)) - h(0.5*(1+sqrt(1-4*p*(1-p)*q**2)))
        return 1/tmp

    res = minimize(calc_XN_tmp, np.array([0.5]), method='nelder-mead', 
                       options={'xtol': 1e-8, 'disp': False});
    return 1/res.fun;


def A_p2(rho): # this is the amplitude dampening channel on 2 qubits (2 uses). rho is a 2-qubit state.

    return  KA * rho * KA_dag + KB * rho * KB_dag + KC * rho * KC_dag + KD * rho * KD_dag 


def Func(concurrences, angles1, angles2, ps):   #This is the estimate of X(N \tensor N)
    
    #----------make the states with the given concurrences
    #XNN = []
#    for i in range(concurrences.shape[0]):
#        first_term = 0
#        second_term = 0
#        for j in range(INPUT_STATES):
#            thetas = arcsin(concurrences)
#            measure = cos(thetas[i,j]/2) * basis(2,0) + sin(thetas[i,j]/2) * basis(2,1)
#            ket = (tensor(measure.dag(), I2) * ghz).unit()
#            ket_rotated = tensor(ry(angles1[i,j] * 2 * pi), ry(angles2[i,j] * 2 * pi)) * ket 
#            first_term = first_term + ps[i,j] * A_p2(ket2dm(ket_rotated))
#            second_term = second_term + ps[i,j] * entropy_vn(A_p2(ket2dm(ket_rotated)), base=2)
#        first_term = entropy_vn(first_term, base=2)
#        XNN.append(first_term - second_term)

    thetas = arcsin(concurrences)
    costhetas = cos(0.5*thetas)
    sinthetas = sin(0.5*thetas)
    c = np.dot(costhetas, bra0I2ghz)
    s = np.dot(sinthetas, bra1I2ghz)
    cs = c + s
    K = [[A_p2(ket2dm(ry(angles1[i,j] * 2 * pi, N=2, target=0) * ry(angles2[i,j] * 2 * pi, N=2, target=1) * cs[i,j])) for j in range(INPUT_STATES)]  
         for i in range(concurrences.shape[0])  ]

    XNN = [ entropy_vn(sum( ps[i,j] * K[i][j]  for j in range(INPUT_STATES)  ) , base=2)  - 
         sum( ps[i,j] * entropy_vn(K[i][j] , base=2)  
                                        for j in range(INPUT_STATES) )    for i in range(concurrences.shape[0])]

    
    return np.array(XNN)
    
# find non-zero fitness for selection.
def get_fitness(pred): return pred + 1e-3 - np.min(pred)


def translateDNA(pop):   #pop is a list of 2 np arrays. One nparray for the population of X and Y, and one nparray for the population of ps
    pop_dec = pop.dot(2 ** np.arange(DNA_SIZE)[::-1]) / (2**DNA_SIZE-1)
    return [pop_dec[:,:,0], pop_dec[:,:,1], pop_dec[:,:,2], 
         pop_dec[:,:,3]/np.sum(pop_dec[:,:,3], axis=1)[:,None] ]  #conc, rot1, rot2, p

# nature selection wrt pop's fitness.
def select(pop, fitness):    
    idx = np.random.choice(np.arange(pop.shape[0]), size=POP_SIZE, replace=True, p=fitness/fitness.sum())
    A = pop[idx]
    A[0] = pop[np.argmax(fitness)]
    return A

# mating process (genes crossover).
def crossover_and_mutate(individual, pop, mutate=False):
    #crossover
    if np.random.rand() < CROSS_RATE:
        i_ = np.random.randint(0, POP_SIZE, size=1)[0]                        # select another individual from pop
        cross_points = np.random.randint(0, 2, size=4*DNA_SIZE*INPUT_STATES).astype(bool).reshape((INPUT_STATES,4,DNA_SIZE))
        individual[cross_points] = pop[i_][cross_points]
        
    #mutate
    if mutate and np.random.rand() < MUTATION_FRACTION:       
        x = np.random.choice([0, 1], size=INPUT_STATES*4*DNA_SIZE, 
                             p=[1-MUTATION_RATE, MUTATION_RATE]).astype(bool).reshape((INPUT_STATES,4,DNA_SIZE))

        individual[x] = np.abs(individual[x] - 1)
        
    return individual


In [ ]:
p = 0.001                 #probability p in the amp. damp. channel
DNA_SIZE = 25           # DNA length   # size of each number in the lists X and Y
POP_SIZE = 10           # population size
CROSS_RATE = 1          # mating probability (DNA crossover)
MUTATION_RATE = 0.005   # mutation probability
MUTATION_FRACTION = 1   #percentage of children to mutate
N_GENERATIONS = 500

#---------------Moved from functions for optimization
K1 = basis(2,0) * basis(2,0).dag() + sqrt(1-p) * basis(2,1) * basis(2,1).dag()
K2 = sqrt(p) * basis(2,0) * basis(2,1).dag()
KA = tensor(K1,K1)
KA_dag = tensor(K1,K1).dag()
KB = tensor(K1,K2)
KB_dag = tensor(K1,K2).dag()
KC = tensor(K2,K1)
KC_dag = tensor(K2,K1).dag()
KD = tensor(K2,K2)
KD_dag = tensor(K2,K2).dag()


I2 = tensor(identity(2), identity(2))

ghz = ghz_state()

bra0 = basis(2).dag()
bra0I2 = tensor(bra0,I2)
bra0I2ghz = (tensor(bra0,I2) * ghz).unit()
bra1 = basis(2,1).dag()
bra1I2 = tensor(bra1,I2)
bra1I2ghz = (tensor(bra1,I2) * ghz).unit()
#-------------------------

pop = np.random.randint(2, size = ([POP_SIZE, INPUT_STATES, 4, DNA_SIZE])) #conc,rot1,rot2,p

#-----------------For plotting
#plt.ion()
fig, ax = plt.subplots()
gen, holevo = [],[]
ax.scatter(gen,holevo)
plt.xlim(0,N_GENERATIONS)
#plt.ylim(0.4,1.5)
XN = calc_XN(p) # this is X(N). You need to plot 2X(N)
plt.plot([i for i in range(N_GENERATIONS)], 
         [2*XN for j in range(N_GENERATIONS)])
plt.draw()
plt.xlabel("Generations")
plt.ylabel("X(N tensor N) estimate")
#-----------------------------

win_flag = 0
win_probs = 0
win_concs = 0
win_rot1 = 0
win_rot2 = 0
for _ in range(N_GENERATIONS):
    #-=----------variable mutation rate
    #if _ > 700: MUTATION_RATE = 0.001
    #---------------------
    translated = translateDNA(pop)
    F_values = Func(translated[0], translated[1], translated[2], translated[3])    # compute function value by extracting DNA
    holevo.append(np.max(F_values))
    if holevo[-1] >= 2*XN:
        win_probs = translated[3][np.argmax(F_values)]
        win_concs = translated[0][np.argmax(F_values)]
        win_rot1 = translated[1][np.argmax(F_values)]
        win_rot2 = translated[2][np.argmax(F_values)]
    #clear_output(wait=False)
    #print("probabilities ", translated[3][np.argmax(F_values)])
    #print("angles1 (2 pi) ", translated[1][np.argmax(F_values)]) 
    #print("angles2 (2 pi) ", translated[2][np.argmax(F_values)])
    #print("concurrences ", translated[0][np.argmax(F_values)])
    
    #-------plot
    ax.scatter(_, holevo[-1], c='red')
    plt.pause(0.005)
    #-------

    # GA part (evolution)
    fitness = get_fitness(F_values) #FIXED
    pop = select(pop, fitness)
    pop_copy = pop.copy()
    pop_copy_2 = pop.copy()
    #first_member_flag = 1   #don't mutate first member (because it has the highest fitness)
    for parent in pop:
        child = crossover_and_mutate(parent, pop_copy, len(holevo)>1 and holevo[-1]==holevo[-2]) 
        #parent[:] = child     # parent is replaced by its child
        pop_copy_2 = np.concatenate((pop_copy_2,child[np.newaxis, ...]),axis=0)
        #first_member_flag +=1
    #first_member_flag = 0
    pop = pop_copy_2


#plt.ioff()

KeyboardInterrupt: 

ERROR:tornado.general:Uncaught exception, closing connection.
Traceback (most recent call last):
  File "C:\Users\basse\Anaconda3\envs\qutip\lib\site-packages\zmq\eventloop\zmqstream.py", line 414, in _run_callback
    callback(*args, **kwargs)
  File "C:\Users\basse\Anaconda3\envs\qutip\lib\site-packages\tornado\stack_context.py", line 277, in null_wrapper
    return fn(*args, **kwargs)
  File "C:\Users\basse\Anaconda3\envs\qutip\lib\site-packages\ipykernel\kernelbase.py", line 283, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "C:\Users\basse\Anaconda3\envs\qutip\lib\site-packages\ipykernel\kernelbase.py", line 233, in dispatch_shell
    handler(stream, idents, msg)
  File "C:\Users\basse\Anaconda3\envs\qutip\lib\site-packages\ipykernel\kernelbase.py", line 421, in execute_request
    self._abort_queues()
  File "C:\Users\basse\Anaconda3\envs\qutip\lib\site-packages\ipykernel\kernelbase.py", line 636, in _abort_queues
    self._abort_queue(stream)
  File "C:\Users\

In [ ]:
print(win_flag)

In [ ]:
print(win_probs)

In [ ]:
print(sum(win_probs))

In [ ]:
print(win_concs)

In [ ]:
print(win_rot1)

In [ ]:
print(win_rot2)